<a href="https://colab.research.google.com/github/carloidangelo/ECSE-415/blob/main/A3_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##Image Classification with Convolution Neural Network (CNN)

In [ ]:
# Download dataset (batch size = 32)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [ ]:
# Implement CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 4096)
        x = self.fc1(x)
        return x

# run code on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net = Net()
net = net.to(device)

cuda:0


In [ ]:
# Create an instance of SGD optimizer with learning rate of 0.001
optimizer = optim.SGD(net.parameters(), lr=0.001)

# Create an instance of categorical cross entropy criterion
criterion = nn.CrossEntropyLoss()

In [ ]:
# Train the CNN for 10 epochs
for epoch in range(10):  # loop over the dataset multiple times

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 625 == 624:    # print every 625 mini-batches
      print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 625))
      running_loss = 0.0

print('Finished Training')

[1,   625] loss: 2.288
[1,  1250] loss: 2.219
[1,  1875] loss: 1.282
[2,   625] loss: 0.497
[2,  1250] loss: 0.427
[2,  1875] loss: 0.378
[3,   625] loss: 0.365
[3,  1250] loss: 0.344
[3,  1875] loss: 0.337
[4,   625] loss: 0.313
[4,  1250] loss: 0.300
[4,  1875] loss: 0.278
[5,   625] loss: 0.264
[5,  1250] loss: 0.245
[5,  1875] loss: 0.230
[6,   625] loss: 0.221
[6,  1250] loss: 0.193
[6,  1875] loss: 0.191
[7,   625] loss: 0.179
[7,  1250] loss: 0.168
[7,  1875] loss: 0.155
[8,   625] loss: 0.149
[8,  1250] loss: 0.145
[8,  1875] loss: 0.134
[9,   625] loss: 0.132
[9,  1250] loss: 0.123
[9,  1875] loss: 0.117
[10,   625] loss: 0.115
[10,  1250] loss: 0.111
[10,  1875] loss: 0.107
Finished Training


In [ ]:
# Predict labels of the test images using the above trained CNN
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Display classification accuracy
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 97 %
